# Clase 1. Ejercicios

(Explicación en 02:40:14)

#### Ejercicio 1

Dado un corpus de documentos representados por una lista de textos, devolver una matriz con la representación one-hot-encoding.


In [3]:
import numpy as np
from itertools import chain

In [4]:
corpus = [ 
    "hola como estas", 
    "hola como estas y como llego al subte",
    "como llego al subte",
    "esta palabra se repite mucho mucho mucho"
]

def get_vocab_from_corpus(corpus):
    return np.unique(list(chain.from_iterable(map(str.split, corpus))))

In [5]:
vocab = get_vocab_from_corpus(corpus)
vocab

array(['al', 'como', 'esta', 'estas', 'hola', 'llego', 'mucho', 'palabra',
       'repite', 'se', 'subte', 'y'], dtype='<U7')

Ejemplo. Ver un documento.

In [6]:
doc1 = corpus[0]
doc1

'hola como estas'

Expresar un documento cómo una lista de índices de un vocabulario.

In [7]:
def get_vocab_indexes(doc,vocab):
    splitted_doc = doc.split()
    return np.array([np.where(vocab==x) for x in splitted_doc]).flatten()

In [8]:
get_vocab_indexes(doc1,vocab)

array([4, 1, 3])

One Hot Encode.

In [9]:
def one_hot_encode_doc(doc,vocab):
    doc_ohe = np.zeros(vocab.size)
    doc_ohe[get_vocab_indexes(doc,vocab)] = 1
    return doc_ohe

In [10]:
def one_hot_encode_corpus(corpus,vocab):
    corpus_ohe = np.zeros(shape=(len(corpus),vocab.size))
    for idx, doc in enumerate(corpus):
        corpus_ohe[idx] = one_hot_encode_doc(doc,vocab)
    return corpus_ohe

corpus_ohe = one_hot_encode_corpus(corpus,vocab)
corpus_ohe

array([[0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0.]])

#### Ejercicio 2

Dado un corpus de documentos representados por una lista de textos, devolver una matriz con la representación vectores de frecuencia.

In [11]:
def freq_encode_doc(doc,vocab):
    doc_freq = np.zeros(vocab.size)
    unique_idx,freq = np.unique(get_vocab_indexes(doc,vocab), return_counts=True)
    doc_freq[unique_idx] = freq
    return doc_freq

In [12]:
doc2 = corpus[3]
doc2
print(doc2)
print(freq_encode_doc(doc2,vocab))

esta palabra se repite mucho mucho mucho
[0. 0. 1. 0. 0. 0. 3. 1. 1. 1. 0. 0.]


In [13]:
def freq_encode_corpus(corpus,vocab):
    corpus_freq = np.zeros(shape=(len(corpus),vocab.size))
    for idx, doc in enumerate(corpus):
        corpus_freq[idx] = freq_encode_doc(doc,vocab)
    return corpus_freq

corpus_freq = freq_encode_corpus(corpus,vocab)
corpus_freq

array([[0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 2., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 3., 1., 1., 1., 0., 0.]])

#### Ejercicio 3

Dado un corpus de documentos representados por una lista de textos, devolver una matriz con la representación TFIDF.

In [14]:
def tfidf(corpus,vocab):
    # Term Frec - Inverse Doc Frec (02:18)
    
    # IDF
    N = len(corpus)
    corpus_ohe = one_hot_encode_corpus(corpus,vocab)
    idf = np.log(N)/np.sum(corpus_ohe,axis=0) 
    
    # TF
    tf = freq_encode_corpus(corpus,vocab)
    
    return tf*idf

tfidf(corpus,vocab)

array([[0.        , 0.46209812, 0.        , 0.69314718, 0.69314718,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.69314718, 0.92419624, 0.        , 0.69314718, 0.69314718,
        0.69314718, 0.        , 0.        , 0.        , 0.        ,
        0.69314718, 1.38629436],
       [0.69314718, 0.46209812, 0.        , 0.        , 0.        ,
        0.69314718, 0.        , 0.        , 0.        , 0.        ,
        0.69314718, 0.        ],
       [0.        , 0.        , 1.38629436, 0.        , 0.        ,
        0.        , 4.15888308, 1.38629436, 1.38629436, 1.38629436,
        0.        , 0.        ]])

#### Ejercicio 4

Escribir una función que reciba la matriz representando el corpus y el índice de un documento y devuelva los documentos ordenados por similitud coseno.

In [15]:
def cosine_similarity(doc1,doc2):
    return np.dot(doc1,doc2)/( np.linalg.norm(doc1) * np.linalg.norm(doc2) )

In [16]:
def sort_docs_by_cosine_similarity(corpus,doc_index):
    vocab = get_vocab_from_corpus(corpus)
    corpus_tf_idf = tfidf(corpus,vocab)
    
    N = len(corpus)
    result = []
    for i in range(N):
        result.append( {
            "doc": corpus[i],    
            "similarity": cosine_similarity(corpus_tf_idf[i],corpus_tf_idf[doc_index])
                
        })    
    return sorted(result, reverse = True, key=lambda e: e['similarity'])

result = sort_docs_by_cosine_similarity(corpus,0)
result

[{'doc': 'hola como estas', 'similarity': 0.9999999999999998},
 {'doc': 'hola como estas y como llego al subte',
  'similarity': 0.5628285773640647},
 {'doc': 'como llego al subte', 'similarity': 0.15316791621349668},
 {'doc': 'esta palabra se repite mucho mucho mucho', 'similarity': 0.0}]

#### Ejercicio 5

Implementar (3) con matrices sparse de la librería SciPy

In [108]:
corpus

['hola como estas',
 'hola como estas y como llego al subte',
 'como llego al subte',
 'esta palabra se repite mucho mucho mucho']

In [119]:
from scipy.sparse import bsr_matrix,csr_matrix
from itertools import chain

def encode_corpus_sparse(corpus):
    splitted_corpus = [x.split() for x in corpus]
    indptr = [0]
    indices = []
    data = []
    vocab = {}
    for doc in splitted_corpus:
        for term in doc:
            index = vocab.setdefault(term, len(vocab))
            indices.append(index)
            data.append(1)
        indptr.append(len(indices))
        print(indices)
        print(data)
    encoded_corpus = csr_matrix((data, indices, indptr), dtype=int)
    return encoded_corpus, vocab

encoded_corpus, vocab = encode_corpus_sparse(corpus)
print(encoded_corpus.toarray())
print(vocab)

[0, 1, 2]
[1, 1, 1]
[0, 1, 2, 0, 1, 2, 3, 1, 4, 5, 6]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 2, 0, 1, 2, 3, 1, 4, 5, 6, 1, 4, 5, 6]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 2, 0, 1, 2, 3, 1, 4, 5, 6, 1, 4, 5, 6, 7, 8, 9, 10, 11, 11, 11]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[[1 1 1 0 0 0 0 0 0 0 0 0]
 [1 2 1 1 1 1 1 0 0 0 0 0]
 [0 1 0 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 3]]
{'hola': 0, 'como': 1, 'estas': 2, 'y': 3, 'llego': 4, 'al': 5, 'subte': 6, 'esta': 7, 'palabra': 8, 'se': 9, 'repite': 10, 'mucho': 11}


In [120]:
#FIXME algo está mal acá. De dónde sale el último 3?